# Minimal example

## Prerequisites

To start, we first import the required classes, enums, and functions from ConFlowGen:

In [ ]:
from conflowgen import ContainerFlowGenerationManager
from conflowgen import ModeOfTransport
from conflowgen import PortCallManager
from conflowgen import ExportFileFormat
from conflowgen import ExportContainerFlowManager
from conflowgen import DatabaseChooser
from conflowgen import setup_logger
from conflowgen import InboundAndOutboundVehicleCapacityPreviewReport
from conflowgen import ContainerFlowByVehicleTypePreviewReport
from conflowgen import VehicleCapacityExceededPreviewReport
from conflowgen import ModalSplitPreviewReport
from conflowgen import InboundAndOutboundVehicleCapacityAnalysisReport
from conflowgen import ContainerFlowByVehicleTypeAnalysisReport
from conflowgen import ModalSplitAnalysisReport
from conflowgen import ContainerFlowAdjustmentByVehicleTypeAnalysisReport
from conflowgen import ContainerFlowAdjustmentByVehicleTypeAnalysisSummaryReport
import datetime
logger = setup_logger()

## Database selection

Now, we select a database to work in. If the selected database already exists within the project folder, it is loaded. Otherwise, it is created.

In [ ]:
database_chooser = DatabaseChooser()
demo_file_name = "my_demo.sqlite"
if demo_file_name in database_chooser.list_all_sqlite_databases():
    database_chooser.load_existing_sqlite_database(demo_file_name)
else:
    database_chooser.create_new_sqlite_database(demo_file_name)

## General settings

In [ ]:
container_flow_generation_manager = ContainerFlowGenerationManager()
container_flow_generation_manager.set_properties(
    name="Demo file",
    start_date=datetime.datetime.now().date(),
    end_date=datetime.datetime.now().date() + datetime.timedelta(days=21)
)

## Creating schedules

In [ ]:
port_call_manager = PortCallManager()

At first we define a name for our new feeder liner service.

In [ ]:
feeder_service_name = "LX050"

In [ ]:
port_call_manager.add_large_scheduled_vehicle(
    vehicle_type=ModeOfTransport.feeder,
    service_name=feeder_service_name,
    vehicle_arrives_at=datetime.date(2021, 7, 9),
    vehicle_arrives_at_time=datetime.time(11),
    average_vehicle_capacity=800,
    average_moved_capacity=100,
    next_destinations=[
        ("DEBRV", 0.4),  # 40% of the containers go here...
        ("RULED", 0.6)   # and the other 60% of the containers go here.
    ]
)

The overall code in the demo for the creation of a feeder service looks like this. Here, the code is wrapped in an if condition to check if the liner service is not already existing and comes with some additional logging information. The logging part may be helpful, but is not explained further here.

In [ ]:
port_call_manager = PortCallManager()
feeder_service_name = "LX050"
if not port_call_manager.has_schedule(feeder_service_name, vehicle_type=ModeOfTransport.feeder):
    logger.info(f"Add feeder service '{feeder_service_name}' to database")
    port_call_manager.add_large_scheduled_vehicle(
        vehicle_type=ModeOfTransport.feeder,
        service_name=feeder_service_name,
        vehicle_arrives_at=datetime.date(2021, 7, 9),
        vehicle_arrives_at_time=datetime.time(11),
        average_vehicle_capacity=800,
        average_moved_capacity=100,
        next_destinations=[
            ("DEBRV", 0.4),  # 40% of the containers go here...
            ("RULED", 0.6)   # and the other 60% of the containers go here.
        ]
    )
else:
    logger.info(f"Feeder service '{feeder_service_name}' already exists")

Following the same principle and structure we can also add schedules for trains and deep sea vessels:

In [ ]:
train_service_name = "JR03A"
if not port_call_manager.has_schedule(train_service_name, vehicle_type=ModeOfTransport.train):
    logger.info(f"Add train service '{train_service_name}' to database")
    port_call_manager.add_large_scheduled_vehicle(
        vehicle_type=ModeOfTransport.train,
        service_name=train_service_name,
        vehicle_arrives_at=datetime.date(2021, 7, 12),
        vehicle_arrives_at_time=datetime.time(17),
        average_vehicle_capacity=90,
        average_moved_capacity=90,
        next_destinations=None  # Here we don't have containers that need to be grouped by destination
    )
else:
    logger.info(f"Train service '{train_service_name}' already exists")

In [ ]:
deep_sea_service_name = "LX050"
if not port_call_manager.has_schedule(deep_sea_service_name, vehicle_type=ModeOfTransport.deep_sea_vessel):
    logger.info(f"Add deep sea vessel service '{deep_sea_service_name}' to database")
    port_call_manager.add_large_scheduled_vehicle(
        vehicle_type=ModeOfTransport.deep_sea_vessel,
        service_name=deep_sea_service_name,
        vehicle_arrives_at=datetime.date(2021, 7, 10),
        vehicle_arrives_at_time=datetime.time(19),
        average_vehicle_capacity=16000,
        average_moved_capacity=150,  # for faster demo
        next_destinations=[
            ("ZADUR", 0.3),  # 30% of the containers go to ZADUR...
            ("CNSHG", 0.7)   # and the other 70% of the containers go to CNSHG.
        ]
    )
else:
    logger.info(f"Deep sea service '{deep_sea_service_name}' already exists")

## Generate the data

In [ ]:
container_flow_generation_manager.generate()